# Re-indexing SHOC files for OceanParcels

In [8]:
import parcels
import xarray as xr
import numpy as np
import datetime as dati
import parcels
import netCDF4
# import matplotlib
# from matplotlib import pylab
from parcels import FieldSet, Field, Variable, Field
from parcels import ParticleSet, ScipyParticle, JITParticle, AdvectionRK4_3D, AdvectionRK4
# from parcels import FieldSamplingError, ParticleFile, plotTrajectoriesFile

In [41]:
in_files = [
    "SHOC_flowfield_files/in_trans_2024-05.nc"
]
coord_file = "RI_flowfield_files/coord_file.nc"

In [42]:
# Load flowfield data into memory
ff = xr.open_dataset(in_files[0], decode_times = False).load()
ff.close()

# View in xarray dataset
print(np.nanmin(ff.botz.values), np.nanmax(ff.botz.values))
print(np.nanmin(ff.x_grid.values), np.nanmax(ff.x_grid.values))
print(np.nanmin(ff.y_grid.values), np.nanmax(ff.y_grid.values))
ff

-25.0549375 0.0
147.327451 147.347847
-43.026218 -43.016867


<xarray.Dataset> Size: 241MB
Dimensions:   (k_grid: 22, k_centre: 21, j_grid: 22, i_grid: 35, j_centre: 21,
               i_centre: 34, ns2: 596, ns3: 10289, record: 361)
Dimensions without coordinates: k_grid, k_centre, j_grid, i_grid, j_centre,
                                i_centre, ns2, ns3, record
Data variables: (12/30)
    z_grid    (k_grid) float64 176B -30.0 -27.0 -22.0 -18.0 ... 0.0 0.2 1e+20
    z_centre  (k_centre) float64 168B -28.5 -24.5 -20.0 -16.5 ... -0.1 0.1 0.3
    x_grid    (j_grid, i_grid) float64 6kB 147.3 147.3 147.3 ... 147.3 147.3
    y_grid    (j_grid, i_grid) float64 6kB -43.03 -43.03 ... -43.02 -43.02
    x_centre  (j_centre, i_centre) float64 6kB 147.3 147.3 147.3 ... 147.3 147.3
    y_centre  (j_centre, i_centre) float64 6kB -43.03 -43.03 ... -43.02 -43.02
    ...        ...
    temp      (record, ns3) float64 30MB 20.0 20.0 20.0 20.0 ... 0.0 0.0 0.0 0.0
    salt      (record, ns3) float64 30MB 35.0 35.0 35.0 35.0 ... 0.0 0.0 0.0 0.0
    Kzmean    (record, ns3) float64 30MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    u1vmean   (record, ns3) float64 30MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    u2vmean   (record, ns3) float64 30MB 0.0 0.0 0.0 0.0 ... 0.00437 0.0 0.0
    swr       (record, ns2) float64 2MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/16)
    title:         SHOC grid
    description:   Automated grid from auto.prm
    paramhead:     75 RunName: Tinderbox_HighRes
    codehead:      SHOC default version
    paramfile:     /g/data/ew0/scott/Tinderbox/75/inputs/in.prm
    ems_version:   v1.5.3
    ...            ...
    nce1:          34
    nce2:          21
    nfe1:          35
    nfe2:          22
    ns3:           10289
    ns2:           596

**From the NEMO tutorial**

NEMO is discretised on a Arakawa C-grid. (U and V velocities are not located on the same nodes, see https://www.nemo-ocean.eu/doc/node19.html). To interpolate U, V velocities on a C-grid, parcels needs to read the f-nodes, which are located on the corners of the cells.

According to NEMO's documentation (https://www.nemo-ocean.eu/doc/img360.png, https://docs.oceanparcels.org/en/latest/examples/documentation_indexing.html#NEMO-Example), NEMO is indexed as follows in the horizontal:
```
 ____V[ij]___.f[ij]
|            |
|            |
|     .T[ij] |U[ij]
|            |
|____________|
```
Where i and j are increasing in the horizontal and vertical directions respectively and T is the location of tracer data. A NEMO cell face will therefore have nodes `[i-1,j-1]`, `[i-1,j]`, `[i,j-1]`, and `[i,j]` forming its bottom-left, top-left, bottom-right, and top-right corners respectively.

By contrast, the SHOC grid is indexed as follows in the horizontal:
```
      ____________
     |            |
     |            |
U[ij]|     .T[ij] |
     |            |
     |.___________|
 f[ij]    V[ij]
```
A SHOC cell face will therefore have nodes `[i,j]`, `[i,j+1]`, `[i+1,j]`, and `[i+1,j+1]` forming its bottom-left, top-left, bottom-right, and top-right corners respectively.

Converting these points to SHOC therefore becomes: `NEMO_F[i,j] = SHOC_F[i+1,j+1]`. At the same time, SHOC provides a bit more grid information, the depths are flipped, and a few other transformations are needed.

In [43]:
# Get the grid coordinates from SHOC
i_grid = ff.x_grid.values[0,:]
j_grid = ff.y_grid.values[:,0]
k_grid = np.flip(ff.z_grid.values) # depth needs to be flipped
botz = ff.botz.values

# Can't use the bottom of each horizontal dim because we don't have T[i-1, j-1]
i_grid = i_grid[1:]
j_grid = j_grid[1:]
k_grid = k_grid[1:]
cells_i = np.size(i_grid)
cells_j = np.size(j_grid)

# Clip depth - EMS uses sediment/wind but we're not doing that
k_grid = k_grid[1:]*-1
k_grid[0] = 0
cells_k = np.size(k_grid)
k_grid = k_grid[0:cells_k-1]
cells_k = np.size(k_grid)

In [44]:
# MAKE THEM GRIDS
lat_grid = np.repeat(j_grid[:, np.newaxis], cells_i, axis = 1).astype('float32')
lon_grid = np.repeat(i_grid[np.newaxis, :], cells_j, axis = 0).astype('float32')

dep_grid = np.repeat(k_grid[:, np.newaxis], cells_j, axis = 1)
dep_grid = np.repeat(dep_grid[:, :, np.newaxis], cells_i, axis = 2).astype('float32')

In [45]:
ds_coords = xr.Dataset(
    data_vars = {
        'lat_grid': (['j','i'], lat_grid),
        'lon_grid': (['j','i'], lon_grid),
        'dep_grid': (['k','j','i'], dep_grid),
        # 'botz': (['j','i'], botz)
    }
)

In [46]:
ds_coords.dep_grid.attrs = {'long_name': 'Vertical depth', 'units': 'm', 'positive': 'down'}
ds_coords.lat_grid.attrs = {'long_name': 'Latitude', 'units': 'degrees_north'}
ds_coords.lon_grid.attrs = {'long_name': 'Longitude', 'units': 'degrees_east'}
# ds_coords.botz.attrs = {'long_name': 'Depth of the benthos', 'units': 'm'}

xr.set_options(keep_attrs = True)
ds_coords.to_netcdf(coord_file)

### Coordinates grid

In the NEMO files, latitude and longitude are stored in a seperate file BUT are both 3D (time dimension, with size=1). Depth is stored seperately and is 1D, and the data is 4D. It's not really necessary to have a seperate coordinates file but it might save some memory, and parcels is expecting it.